# Where have I been?
[WHIB](http://www.bleatinc.com/) is constantly running on my iPhone and tracks my location with minimal battery impact.
This notebook here parses and display the exported data[^1] from the app into something pretty.

[^1]: A 2$/month *premium* feature lets you export all the data as CSV, which is easy to parse and display.

In [8]:
import glob
import pandas
import geopy
import geopy.distance
import geopy.geocoders
from geopy.extra.rate_limiter import RateLimiter
import folium
import folium.plugins

In [3]:
# Which year do we want to look at?
whichyear = 2021

In [5]:
# Tile providers
# Available tiles: https://github.com/python-visualization/folium/blob/master/folium/folium.py#L75
#tileprovider = 'Mapbox Bright'
tileprovider = 'Cartodb positron'
#tileprovider = 'Cartodb dark_matter'
#tileprovider = 'Mapbox Control Room'
#tileprovider = 'Stamen Toner'

In [21]:
# Default zoom level and circle marker radius for map
zoom_start = 5
radius = 10

In [22]:
# Settings for address lookup
# Set an unique user agent
geopy.geocoders.options.default_user_agent = 'Jahresrückblick Habi. Contact habi@gna.ch if you have an issue with me!'
# Be patient
geolocator = geopy.geocoders.Nominatim(timeout=5)
# Don't be too needy
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [23]:
# Read in locations from newest CSV-file in the current directory
locations = pandas.read_csv(sorted(glob.glob('journey*.csv'))[-1])

In [24]:
# Modify the dataframe
locations.drop(['Crumb'], axis=1, inplace=True)
locations.rename(columns={'LocalDate': 'Date'}, inplace=True)
locations.rename(columns={'LocalTime': 'Time'}, inplace=True)
locations.rename(columns={'Altitude (in metres)': 'Altitude'}, inplace=True)
locations.rename(columns={'Accuracy (in metres)': 'Accuray'}, inplace=True)

In [25]:
# Make us a proper date column, based on https://stackoverflow.com/a/26763793
locations['Date'] = pandas.to_datetime(locations['Date'])
# Make us a year, month and weekday colum, based on https://stackoverflow.com/q/48623332
locations['Year'] = locations.Date.dt.year
locations['Month'] = locations.Date.dt.month
locations['Day'] = locations.Date.dt.day
locations['Weekday'] = locations.Date.dt.dayofweek

In [26]:
len(locations)

62484

In [27]:
# Drop all values not in 'whichyear'
# https://stackoverflow.com/a/27360130
locations.drop(locations[locations.Year != whichyear].index, inplace=True)
# Reset index, so that we can find the correct date lateron
locations.reset_index(drop=True, inplace=True)

In [28]:
len(locations)

16871

In [29]:
# Drop all values not in a certain month
# locations.drop(locations[locations.Month != 7].index, inplace=True)

In [30]:
locations.head()

,Date,Time,Latitude,Longitude,Altitude,Accuray,Year,Month,Day,Weekday
0,2021-01-01,15:34,46.935399,7.417817,555,22,2021,1,1,4
1,2021-01-01,15:34,46.935403,7.421076,547,65,2021,1,1,4
2,2021-01-01,15:35,46.935023,7.422048,548,65,2021,1,1,4
3,2021-01-01,15:36,46.934322,7.423566,553,65,2021,1,1,4
4,2021-01-01,15:36,46.934149,7.424638,555,65,2021,1,1,4


In [31]:
locations.tail()

,Date,Time,Latitude,Longitude,Altitude,Accuray,Year,Month,Day,Weekday
16866,2021-12-31,18:32,46.935573,7.418067,0,0,2021,12,31,4
16867,2021-12-31,22:12,46.935284,7.417968,561,26,2021,12,31,4
16868,2021-12-31,22:21,46.937574,7.420472,550,42,2021,12,31,4
16869,2021-12-31,22:23,46.935573,7.418067,0,0,2021,12,31,4
16870,2021-12-31,22:24,46.935458,7.418326,550,30,2021,12,31,4


In [32]:
# Show the extreme locations on a map
# Marker colors from here: https://stackoverflow.com/a/41993318
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
# Altitude
for c, loc in locations.sort_values(by=['Altitude'],
                                    ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='Highest: %s' % (loc.Date),
                       ).add_to(m)
# North
for c, loc in locations.sort_values(by=['Latitude'],
                                    ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='North: %s' % (loc.Date),
                       ).add_to(m)
# East
for c, loc in locations.sort_values(by=['Longitude'],
                                    ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='East: %s' % (loc.Date),
                       ).add_to(m)
# South
for c, loc in locations.sort_values(by=['Latitude']).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='South: %s' % (loc.Date),
                       ).add_to(m)
# West
for c, loc in locations.sort_values(by=['Longitude']).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='West: %s' % (loc.Date),
                       ).add_to(m)
m

In [33]:
# Show north, east, south, west extreme (with averaged lat/lon for each)
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
                tiles=tileprovider,
                zoom_start=zoom_start)
folium.CircleMarker(location=[locations.Latitude.mean(),
                              locations.Longitude.min()],
                    radius=radius,
                    popup='Average West').add_to(m)
folium.CircleMarker(location=[locations.Latitude.mean(),
                              locations.Longitude.max()],
                    radius=radius,
                    popup='Average East').add_to(m)
folium.CircleMarker(location=[locations.Latitude.max(),
                              locations.Longitude.mean()],
                    radius=radius,
                    popup='Average North').add_to(m)
folium.CircleMarker(location=[locations.Latitude.min(),
                              locations.Longitude.mean()],
                    radius=radius,
                    popup='Average South').add_to(m)
m

In [34]:
# How far did we come?
print('In %s, we traveled %0.0f km north-south' % (whichyear,
                                                   geopy.distance.geodesic([locations.Latitude.min(),
                                                                            locations.Longitude.mean()],
                                                                         [locations.Latitude.max(),
                                                                          locations.Longitude.mean()]).km))
print('In %s, we traveled %0.0f km east-west' % (whichyear,
                                                 geopy.distance.geodesic([locations.Latitude.mean(),
                                                                          locations.Longitude.min()],
                                                                         [locations.Latitude.mean(),
                                                                          locations.Longitude.max()]).km))


In 2021, we traveled 1364 km north-south
In 2021, we traveled 1579 km east-west


By using [the geocoding library for Python](https://github.com/geopy/geopy) and the [OpenStreetMap reverse geocoding tool](https://wiki.openstreetmap.org/wiki/Nominatim) we can assign addresses to locations.

In [35]:
# Get all extrema locations
location_average = [locations['Latitude'].mean(), locations['Longitude'].mean()]
location_median = [locations['Latitude'].median(), locations['Longitude'].median()]
location_north = [locations[locations.Latitude == locations.Latitude.max()].Latitude.values[0],
                  locations[locations.Latitude == locations.Latitude.max()].Longitude.values[0]]
location_east = [locations[locations.Longitude == locations.Longitude.max()].Latitude.values[0],
                 locations[locations.Longitude == locations.Longitude.max()].Longitude.values[0]]
location_south = [locations[locations.Latitude == locations.Latitude.min()].Latitude.values[0],
                  locations[locations.Latitude == locations.Latitude.min()].Longitude.values[0]]
location_west = [locations[locations.Longitude == locations.Longitude.min()].Latitude.values[0],
                 locations[locations.Longitude == locations.Longitude.min()].Longitude.values[0]]
location_top = [locations[locations['Altitude'] == locations['Altitude'].max()].Latitude.values[0],
                locations[locations['Altitude'] == locations['Altitude'].max()].Longitude.values[0]]

In [36]:
# Turn debug on to print and thus find the right value of the address
debug = True

In [37]:
# According to https://github.com/openstreetmap/Nominatim/issues/885#issuecomment-358123829
# we have to
# > Look for the first of 'city', 'town', 'village', 'hamlet', 'suburb'
# Let's do this!
potentialplacename = ['city', 'town', 'village', 'hamlet', 'suburb']

In [38]:
# Address lookup with `geopy` and https://wiki.openstreetmap.org/wiki/Nominatim
location_average_geo = geolocator.reverse(location_average)
# Print the details, so we can get the correct value to save below
if debug:
    print(location_average_geo.raw)
# Save us a name and print it
for p in potentialplacename:
    if location_average_geo.raw.get('address').get(p):
        name_average = location_average_geo.raw.get('address').get(p)
print('The average location in %s was in %s' % (whichyear, name_average))
# Show the point on a map
m = folium.Map(location=[float(location_average_geo.raw.get('lat')),
                         float(location_average_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_average_geo.raw.get('lat')),
                              float(location_average_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_average).add_to(m)
m

{'place_id': 141185499, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 170270269, 'lat': '46.59096577688851', 'lon': '7.716252801577114', 'display_name': 'Eymätteliweg, Kiental, Reichenbach im Kandertal, Verwaltungskreis Frutigen-Niedersimmental, Verwaltungsregion Oberland, Bern/Berne, 3723, Schweiz/Suisse/Svizzera/Svizra', 'address': {'road': 'Eymätteliweg', 'village': 'Kiental', 'city': 'Reichenbach im Kandertal', 'county': 'Verwaltungskreis Frutigen-Niedersimmental', 'state_district': 'Verwaltungsregion Oberland', 'state': 'Bern/Berne', 'postcode': '3723', 'country': 'Schweiz/Suisse/Svizzera/Svizra', 'country_code': 'ch'}, 'boundingbox': ['46.5874651', '46.5913867', '7.714625', '7.7227854']}
The average location in 2021 was in Kiental


In [39]:
location_median_geo = geolocator.reverse(location_median)
if debug:
    print(location_median_geo.raw)
for p in potentialplacename:
    if location_median_geo.raw.get('address').get(p):
        name_median = location_median_geo.raw.get('address').get(p)
print('The median location in %s was in %s' % (whichyear, name_median))
m = folium.Map(location=[float(location_median_geo.raw.get('lat')),
                         float(location_median_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_median_geo.raw.get('lat')),
                              float(location_median_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_median).add_to(m)
m

{'place_id': 115850336, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 52435561, 'lat': '46.935710799999995', 'lon': '7.4252227180351955', 'display_name': 'Spielplatz Steinhölzli, Schwarzenburgstrasse, Weissenbühl, Liebefeld, Stadtteil III, Bern, Verwaltungskreis Bern-Mittelland, Verwaltungsregion Bern-Mittelland, Bern/Berne, 3008, Schweiz/Suisse/Svizzera/Svizra', 'address': {'leisure': 'Spielplatz Steinhölzli', 'road': 'Schwarzenburgstrasse', 'quarter': 'Weissenbühl', 'suburb': 'Liebefeld', 'city_district': 'Stadtteil III', 'city': 'Bern', 'county': 'Verwaltungskreis Bern-Mittelland', 'state_district': 'Verwaltungsregion Bern-Mittelland', 'state': 'Bern/Berne', 'postcode': '3008', 'country': 'Schweiz/Suisse/Svizzera/Svizra', 'country_code': 'ch'}, 'boundingbox': ['46.9352914', '46.9361369', '7.4247927', '7.4255476']}
The median location in 2021 was in Liebefeld


In [40]:
## Get day of location, based on 
# https://stackoverflow.com/a/53979441/323100
# and 
# https://strftime.org
# print(locations[locations.eq(location_north[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d'))

In [41]:
location_north_geo = geolocator.reverse(location_north)
if debug:
    print(location_north_geo.raw)
for p in potentialplacename:
    if location_north_geo.raw.get('address').get(p):
        name_north = location_north_geo.raw.get('address').get(p)
print('The northmost location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_north,
                        location_north_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_north[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_north_geo.raw.get('lat')),
                         float(location_north_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_north_geo.raw.get('lat')),
                              float(location_north_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_north).add_to(m)
m

{'place_id': 207680960, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 482924029, 'lat': '47.59970285', 'lon': '7.531614370371241', 'display_name': 'Euroairport Basel-Mulhouse, Route de Service, Saint-Louis, Mulhouse, Haut-Rhin, Grand Est, France métropolitaine, 68300, France', 'address': {'aeroway': 'Euroairport Basel-Mulhouse', 'road': 'Route de Service', 'town': 'Saint-Louis', 'municipality': 'Mulhouse', 'county': 'Haut-Rhin', 'state': 'Grand Est', 'region': 'France métropolitaine', 'postcode': '68300', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['47.5987823', '47.600514', '7.5307111', '7.5325932']}
The northmost location in 2021 was in Saint-Louis, France on July, 10


In [42]:
location_east_geo = geolocator.reverse(location_east)
if debug:
    print(location_east_geo.raw)
for p in potentialplacename:
    if location_east_geo.raw.get('address').get(p):
        name_east = location_east_geo.raw.get('address').get(p)
print('The most eastern location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_east,
                        location_east_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_east[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_east_geo.raw.get('lat')),
                         float(location_east_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_east_geo.raw.get('lat')),
                              float(location_east_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_east).add_to(m)
m

{'place_id': 196410726, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 419770973, 'lat': '35.3318733', 'lon': '25.2073573', 'display_name': 'Parasiris, Ηρακλείου - Αγίου Νικολάου, Κοινότητα Ελαίας, Δημοτική Ενότητα Γουβών, Δήμος Χερσονήσου, Περιφερειακή Ενότητα Ηρακλείου, Περιφέρεια Κρήτης, Αποκεντρωμένη Διοίκηση Κρήτης, 71601, Ελλάς', 'address': {'amenity': 'Parasiris', 'road': 'Ηρακλείου - Αγίου Νικολάου', 'city_district': 'Κοινότητα Ελαίας', 'city': 'Δημοτική Ενότητα Γουβών', 'municipality': 'Δήμος Χερσονήσου', 'county': 'Περιφερειακή Ενότητα Ηρακλείου', 'state_district': 'Περιφέρεια Κρήτης', 'state': 'Αποκεντρωμένη Διοίκηση Κρήτης', 'postcode': '71601', 'country': 'Ελλάς', 'country_code': 'gr'}, 'boundingbox': ['35.3318033', '35.3319433', '25.2072846', '25.20743']}
The most eastern location in 2021 was in Δημοτική Ενότητα Γουβών, Ελλάς on July, 10


In [43]:
location_south_geo = geolocator.reverse(location_south)
if debug:
    print(location_south_geo.raw)
for p in potentialplacename:
    if location_south_geo.raw.get('address').get(p):
        name_south = location_south_geo.raw.get('address').get(p)
print('The southmost location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_south,
                        location_south_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_south[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_south_geo.raw.get('lat')),
                         float(location_south_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_south_geo.raw.get('lat')),
                              float(location_south_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_south).add_to(m)
m

{'place_id': 116141798, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 55113604, 'lat': '35.31876010955465', 'lon': '25.072200116073383', 'display_name': 'Κοινότητα Γαζίου-Καβροχωρίου, Δημοτική Ενότητα Γαζίου, Δήμος Μαλεβιζίου, Περιφερειακή Ενότητα Ηρακλείου, Περιφέρεια Κρήτης, Αποκεντρωμένη Διοίκηση Κρήτης, 71005, Ελλάς', 'address': {'city_district': 'Κοινότητα Γαζίου-Καβροχωρίου', 'city': 'Δημοτική Ενότητα Γαζίου', 'municipality': 'Δήμος Μαλεβιζίου', 'county': 'Περιφερειακή Ενότητα Ηρακλείου', 'state_district': 'Περιφέρεια Κρήτης', 'state': 'Αποκεντρωμένη Διοίκηση Κρήτης', 'postcode': '71005', 'country': 'Ελλάς', 'country_code': 'gr'}, 'boundingbox': ['35.3186038', '35.3227685', '25.0721326', '25.0727202']}
The southmost location in 2021 was in Δημοτική Ενότητα Γαζίου, Ελλάς on July, 10


In [44]:
location_west_geo = geolocator.reverse(location_west)
if debug:
    print(location_west_geo.raw)
for p in potentialplacename:
    if location_west_geo.raw.get('address').get(p):
        name_west = location_west_geo.raw.get('address').get(p)
print('The most western location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_west,
                        location_west_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_west[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_west_geo.raw.get('lat')),
                         float(location_west_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_west_geo.raw.get('lat')),
                              float(location_west_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_west).add_to(m)
m

{'place_id': 44990469, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 3695767773, 'lat': '44.3218283', 'lon': '4.5439443', 'display_name': "Grotte Chabot, D 290, Sauze, Saint-Martin-d'Ardèche, Privas, Ardèche, Auvergne-Rhône-Alpes, France métropolitaine, 07700, France", 'address': {'natural': 'Grotte Chabot', 'road': 'D 290', 'hamlet': 'Sauze', 'village': "Saint-Martin-d'Ardèche", 'municipality': 'Privas', 'county': 'Ardèche', 'state': 'Auvergne-Rhône-Alpes', 'region': 'France métropolitaine', 'postcode': '07700', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['44.3217783', '44.3218783', '4.5438943', '4.5439943']}
The most western location in 2021 was in Sauze, France on October, 8


In [45]:
location_top_geo = geolocator.reverse(location_top)
if debug:
    print(location_top_geo.raw)
for p in potentialplacename:
    if location_top_geo.raw.get('address').get(p):
        name_top = location_top_geo.raw.get('address').get(p)
print('The highest location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_top,
                        location_top_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_top[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_top_geo.raw.get('lat')),
                         float(location_top_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_top_geo.raw.get('lat')),
                              float(location_top_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_top).add_to(m)
m

{'place_id': 2277682, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 354498113, 'lat': '46.0682488', 'lon': '7.9153252', 'display_name': 'Felskinn, Hinterallalingrat, Saas-Fee, Visp, Valais/Wallis, 3905, Schweiz/Suisse/Svizzera/Svizra', 'address': {'railway': 'Felskinn', 'road': 'Hinterallalingrat', 'village': 'Saas-Fee', 'county': 'Visp', 'state': 'Valais/Wallis', 'postcode': '3905', 'country': 'Schweiz/Suisse/Svizzera/Svizra', 'country_code': 'ch'}, 'boundingbox': ['46.0632488', '46.0732488', '7.9103252', '7.9203252']}
The highest location in 2021 was in Saas-Fee, Schweiz/Suisse/Svizzera/Svizra on July, 25


In [46]:
# Show the extreme values in the overview below or not
showextremes = True

In [47]:
# Plot *all* locations on a single map
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
if showextremes:
    # Extreme locations
    folium.CircleMarker(location=location_average,
                        radius=radius,
                        popup='Average location in %s' % name_average).add_to(m)
    folium.CircleMarker(location=location_north,
                        radius=radius,
                        popup='Northmost location in %s' % name_north).add_to(m)
    folium.CircleMarker(location=location_east,
                        radius=radius,
                        popup='Northmost location in %s' % name_east).add_to(m)
    folium.CircleMarker(location=location_south,
                        radius=radius,
                        popup='Northmost location in %s' % name_south).add_to(m)
    folium.CircleMarker(location=location_west,
                        radius=radius,
                        popup='Northmost location in %s' % name_west).add_to(m)
    folium.CircleMarker(location=location_top,
                        radius=radius,
                        popup='Highest location (%s m) on the %s' % (locations[locations['Altitude'] == locations['Altitude'].max()].Altitude.values[0],
                                                                     name_top)).add_to(m)
# All locations, in different ways
singlepoints = False
fast = True
if singlepoints:
    for c, loc in locations.iterrows():
        # not every point, but every x-th one, or else the map is too slow
        if not c % 10:
            folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                                radius=2,
                                popup='%s@%s' % (loc.Date, loc.Time),
                                color='darkred'
                               ).add_to(m)
else:
    if fast:
        # FastMarkerCluster
        m.add_child(folium.plugins.FastMarkerCluster(locations[['Latitude', 'Longitude']].values.tolist()))
    else:
        # Markercluster
        mc = folium.plugins.MarkerCluster()
        for c, loc in locations.iterrows():
            mc.add_child(folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                                             popup='%s@%s' % (loc.Date, loc.Time)))
        m.add_child(mc)
m.save('map-points.html')
m

In [48]:
# Viridis colormap from here: https://www.thedataschool.co.uk/gwilym-lockwood/viridis-colours-tableau/
gradient={0.00: '#440154FF',
          0.05: '#481567FF',
          0.10: '#482677FF',
          0.15: '#453781FF',
          0.20: '#404788FF',
          0.25: '#39568CFF',
          0.30: '#33638DFF',
          0.35: '#2D708EFF',
          0.40: '#287D8EFF',
          0.45: '#238A8DFF',
          0.50: '#1F968BFF',
          0.55: '#20A387FF',
          0.60: '#29AF7FFF',
          0.65: '#3CBB75FF',
          0.70: '#55C667FF',
          0.75: '#73D055FF',
          0.80: '#95D840FF',
          0.85: '#B8DE29FF',
          0.90: '#DCE319FF',
          0.95: '#FDE725FF'}

In [49]:
# Show a heatmap instead of single points
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
showextremes=False
if showextremes:
    # Extreme locations
    folium.CircleMarker(location=location_average,
                        radius=radius,
                        popup='Average location in %s' % name_average,
                       ).add_to(m)
    folium.CircleMarker(location=location_north,
                        radius=radius,
                        popup='Northmost location in %s' % name_north,
                       ).add_to(m)
    folium.CircleMarker(location=location_east,
                        radius=radius,
                        popup='Most eastern location in %s' % name_east,
                       ).add_to(m)
    folium.CircleMarker(location=location_south,
                        radius=radius,
                        popup='Southmost location in %s' % name_south,
                       ).add_to(m)
    folium.CircleMarker(location=location_west,
                        radius=radius,
                        popup='Most western location in %s' % name_west,
                       ).add_to(m)
    folium.CircleMarker(location=location_top,
                        radius=radius,
                        popup='Highest location (%s m) on the %s' % (locations[locations['Altitude'] == locations['Altitude'].max()].Altitude.values[0],
                                                                     name_top),
                       ).add_to(m)
# Add heatmap
folium.plugins.HeatMap(locations[['Latitude', 'Longitude']].values.tolist(),
                       gradient=gradient).add_to(m)
m.save('map-heat.html')
m